In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：自动学习表格分类模型用于在线解释

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_classification_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_classification_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 查看在 GitHub 上
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_classification_online_explain.ipynb">
      在 Google Cloud Notebooks 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK创建表格分类模型，并使用Google Cloud的AutoML模型进行在线预测并提供解释。详细步骤请参考[AutoML用户指南](https://cloud.google.com/vertex-ai/docs/start/automl-users)。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/iris)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/overview)。这个数据集不需要任何特征工程。您在本教程中将使用的数据集版本存储在公共云存储桶中。训练好的模型可以预测三种类别（setosa、virginica、versicolor）中的鸢尾花品种类型。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本创建一个AutoML表格分类模型，并部署用于在线预测，同时具有可解释性。您也可以选择使用`gcloud`命令行工具或在Cloud Console上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex`Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到服务`Endpoint`资源。
- 使用可解释性进行预测请求。
- 撤销模型部署。

### 费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成费用估算。

### 设置您的本地开发环境

如果您正在使用Colab或谷歌云笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储 SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境的云存储指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一组简洁的指令：

1. [安装和初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装最新版本的Python Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的GA版本*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

安装最新的*Tabulate*库GA版本。

In [ ]:
! pip3 install -U tabulate $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装了额外的软件包后，您需要重新启动笔记本内核，以便它能够找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

开始之前

GPU 运行时

本教程不需要使用 GPU 运行时。

设置您的 Google 云项目

**无论您使用何种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。首次建立帐户时，您将获得 $300 的免费信用额度用于计算和存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行这个笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意：** Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并且使用以 `$` 为前缀的 Python 变量插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量用于本笔记本之余的操作。以下是Vertex AI支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不会使用多区域存储桶来进行Vertex AI训练。并非所有地区都支持所有Vertex AI服务。

了解有关[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试账户或项目。为了避免在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击权限下拉列表。在过滤框中输入“Vertex”，然后选择**Vertex管理员**。在过滤框中输入“存储对象管理员”，然后选择**存储对象管理员**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务帐号密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在各个会话中保留的地方。

请在下面设置您的云存储桶的名称。桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

设定变量

接下来，设定一些在教程中使用的变量。
导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化用于Python的Vertex SDK

为您的项目和相应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在您已经准备好开始创建您自己的AutoML表格分类模型。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/tables/iris_1000.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的鸢尾花数据集的一个版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数(`wc -l`)来计算示例的数量，然后查看前几行。

您还需要了解训练过程中标签列的标题名称，该名称保存为`label_column`。对于这个数据集，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `bq_source`：或者，从BigQuery表中导入数据项到`Dataset`资源。

这个操作可能需要几分钟时间。

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="Iris" + "_" + TIMESTAMP, gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

### 创建并运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行这个管道。

#### 创建训练管道

使用 `AutoMLTabularTrainingJob` 类创建一个AutoML训练管道，参数包括：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `optimization_prediction_type`：为模型训练指定的任务类型。
  - `分类`：一个表格分类模型。
  - `回归`：一个表格回归模型。
- `column_transformations`：（可选）应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练管道的DAG（有向无环图）。

In [ ]:
dag = aip.AutoMLTabularTrainingJob(
    display_name="iris_" + TIMESTAMP,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(dag)

#### 运行训练管道

接下来，通过调用方法`run`以以下参数运行DAG来启动训练作业：

- `dataset`：要训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留置数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签进行训练的列的名称。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，服务可能认为无法进一步改善模型目标测量，则在完全使用预算之前可能会完成训练。

`run`方法完成后将返回`Model`资源。

执行训练管道将需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="iris_" + TIMESTAMP,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 查看模型评估分数
在模型训练完成后，您可以查看其评估分数。

首先，您需要获得对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，也可以列出您项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=iris_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用`deploy`方法，并指定以下参数：

- `machine_type`：计算机的类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送一个带有可解释性的在线预测请求

向部署的模型发送一个带有可解释性的在线预测。在这种方法中，预测的响应将包括一个解释，说明特征如何对结果的解释做出贡献。

制作测试数据项

您将使用合成数据作为测试数据项。不要担心我们使用合成数据——我们只是想展示如何进行预测。

In [ ]:
INSTANCE = {
    "petal_length": "1.4",
    "petal_width": "1.3",
    "sepal_length": "5.1",
    "sepal_width": "2.8",
}

现在您的“Model”资源已部署到“Endpoint”资源，您可以通过将预测请求发送到“Endpoint”资源来进行在线解释。

#### 请求

每个实例的格式为：

    [feature_list]

由于explain()方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

explain()调用的响应是一个带有以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类标签的类名。
- `confidences`：对于分类，每个类标签的预测置信度，介于0和1之间。
- `values`：对于回归，预测值。
- `deployed_model_id`：执行预测的部署的“Model”资源的Vertex AI标识符。
- `explanations`：特征归因。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.explain(instances_list)
print(prediction)

### 理解解释响应

首先，您将查看您的模型预测并将其与实际值进行比较。

In [ ]:
import numpy as np

try:
    label = np.argmax(prediction[0][0]["scores"])
    cls = prediction[0][0]["classes"][label]
    print("Predicted Value:", cls, prediction[0][0]["scores"][label])
except:
    pass

### 检查特征归因

接下来，您将查看此特定示例的特征归因。正面的归因值意味着特定特征使您的模型预测提升了该数量，而负面的归因值则相反。

In [ ]:
from tabulate import tabulate

feature_names = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
attributions = prediction.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, INSTANCE[val], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

### 检查您的解释和基线

为了更好地理解您得到的特征归因，您应该将它们与模型的基线进行比较。在大多数情况下，您的归因值的总和加上基线应该非常接近于模型对每个输入的预测值。同时请注意，对于回归模型，从AI解释返回的`baseline_score`将对发送到您的模型的每个示例都是相同的。对于分类模型，每个类别将有自己的基线。

在此部分，您将向您的模型发送10个测试示例以进行预测，以便将特征归因与基线进行比较。然后，您将通过`sanity_check_explanations`方法来对每个测试示例的归因进行检查。

#### 获取解释

In [ ]:
import random

# Prepare 10 test examples to your model for prediction using a random distribution to generate
# test instances
instances = []
for i in range(10):
    pl = str(random.uniform(1.0, 2.0))
    pw = str(random.uniform(1.0, 2.0))
    sl = str(random.uniform(4.0, 6.0))
    sw = str(random.uniform(2.0, 4.0))
    instances.append(
        {"petal_length": pl, "petal_width": pw, "sepal_length": sl, "sepal_width": sw}
    )

response = endpoint.explain(instances)

#### 合理性检查

在下面的函数中，您对解释进行了合理性检查。

In [ ]:
import numpy as np


def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

## 下架模型

当您完成预测后，您可以从`Endpoint`资源中下架模型。这将取消所有计算资源并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME